This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [328]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [329]:
# First, import the relevant modules
import requests
import collections

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

## Answers 

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.

In [330]:
# Create function to handle stock lookup
def get_StockData(db,symbol,start_date,end_date):
    '''
    Return stock data from a ticker as a dictionary
    '''
    base_url = 'https://www.quandl.com/api/v3/datasets/'
    # Append all of this together
    url = base_url + db + '/' + symbol + '/' + 'data.json?api_key=' + API_KEY + '&start_date=' + start_date + '&end_date=' + end_date
    # Send back Json as a dictionary
    return requests.get(url).json()

In [331]:
# Obtain a single element
def get_Element(stock_data, col_name, date):
    '''
    Get a single element from the data
    '''
    # Get list of data columns
    col_names = stock_data['dataset_data']['column_names']
    # Get all the dates which are in the first index of each data element
    dates = [x[0] for x in stock_data['dataset_data']['data']]
    # Find which column index matches col name
    col_index = col_names.index(col_name)
    # Find which date matches in dates
    date_index = dates.index(date)
    # Index by position
    return stock_data['dataset_data']['data'][date_index][col_index]

In [332]:
# Obtain a data series by name
def get_Series(stock_data, col_name):
    '''
    Get a single element from the data
    '''
    values = []
    # Get list of data columns
    col_names = stock_data['dataset_data']['column_names']
    # Find which column index matches col name
    col_index = col_names.index(col_name)
    # 
    for x in range(0,len(stock_data['dataset_data']['data'])-1):
        if x: 
            temp = stock_data['dataset_data']['data'][x][col_index]
            if temp: # If this is actually a number
                values.append(temp)
    return values

In [333]:
# Pull data in question
data = get_StockData('FSE','AFX_X','2017-01-01','2017-12-31')
type(data)

dict

3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [334]:
max(get_Series(data,'Open'))

53.11

In [335]:
min(get_Series(data,'Open'))

34.0

4. What was the largest change in any one day (based on High and Low price)?

In [336]:
highs = get_Series(data,'High')
lows = get_Series(data,'Low')

max_chg = 0
for x in range(0,len(highs)-1): # for high length
    chg = highs[x] - lows[x] # Find current high and low delta
    if abs(chg) > max_chg: # if the absolute value of that change is higher that max_chg
        max_chg = chg # make Max_chg
round(max_chg,2)

2.81

5. What was the largest change between any two days (based on Closing Price)?

In [337]:
closes = get_Series(data,'Close')
round(max(closes) - min(closes),2)

19.03

6. What was the average daily trading volume during this year?

In [338]:
volume = get_Series(data,'Traded Volume')
round(sum(volume) / len(volume),2)

89515.28

7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [339]:
volume = get_Series(data,'Traded Volume')

def get_Median(values):
    '''
    Get median of a list
    '''
    values.sort() # Sort Values
    leng = len(values) # Get length
    odd = bool(leng % 2) # Is this odd?
    half = leng // 2 # Integer halfway point 
    if leng: # Not empty
        if odd: # Odd case
            median = values[half] # Get integer divided index
        else: # Even Case
            median = (values[half-1] + values[half])/2  
    else: # Empty Case
        median = None

    return median
print(get_Median(volume))

76600.0
